<a href="https://colab.research.google.com/github/willianpsantos/gpu-processing/blob/main/gpu_formula_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 553 (delta 179), reused 147 (delta 100), pack-reused 269 (from 1)
Receiving objects: 100% (553/553), 178.44 KiB | 1.01 MiB/s, done.
Resolving deltas: 100% (281/281), done.


In [2]:
!pip3 install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!python3 /content/rapidsai-csp-utils/colab/pip-install.py

Installing RAPIDS remaining 24.10.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.5/915.5 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 126.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.9/196.9 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.6/836.6 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 157.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.7/722.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.0/916.0 kB 23.3 MB/s e

In [39]:
import cudf as pd
import numpy as np
from numba import cuda

df = pd.read_csv('/content/gpu_data/MAIN_TABLE_TO_PROCESS.csv', ';')

ImportError: cannot import name 'string_view' from 'numba' (/usr/local/lib/python3.10/dist-packages/numba/__init__.py)

In [ ]:
df_additional_measures = pd.read_csv('/content/gpu_data/ADDITIONAL_MEASURES.csv', ';')
df_additional_tw_measures = pd.read_csv('/content/gpu_data/ADDITIONAL_TOWERCO_MEASURES.csv', ';')
df_exchange_rates = pd.read_csv('/content/gpu_data/EXCHANGE_RATE.csv', ';')
df_general_measures = pd.read_csv('/content/gpu_data/GENERAL_MEASURES.csv', ';')
df_leg_for_lookup_codes = pd.read_csv('/content/gpu_data/LEGEND_FOR_LOOKUP_CODES.csv', ';')

joint_gm_names = ','.join(list(df_general_measures['MEASURE_TYPE'].astype(str).values_host))
joint_gm_values = ','.join(list(df_general_measures['MEASURE_VALUE'].astype(str).values_host))
joint_am_names = ','.join(list(df_additional_measures['MEASURE_TYPE'].astype(str).values_host))
joint_am_values = ','.join(list(df_additional_measures['MEASURE_VALUE'].astype(str).values_host))
joint_tw_names = ','.join(list(df_additional_tw_measures['MEASURE_TYPE'].astype(str).values_host))
joint_tw_values = ','.join(list(df_additional_tw_measures['MEASURE_VALUE'].astype(str).values_host))

df['general_measures_names'] = joint_gm_names
df['general_measures_values'] = joint_gm_values
df['additional_measures_names'] = joint_am_names
df['additional_measures_values'] = joint_am_values
df['additional_tw_measures_names'] = joint_tw_names
df['additional_tw_measures_values'] = joint_tw_values



def replace_formula_usd_kernel(row):
  replaced_formula = row['Formula_USD']

  general_measures_names = list(row['general_measures_names'])
  general_measures_values = list(row['general_measures_values'])
  additional_measures_names = list(row['additional_measures_names'])
  additional_measures_values = list(row['additional_measures_values'])
  additional_tw_measures_names = list(row['additional_tw_measures_names'])
  additional_tw_measures_values = list(row['additional_tw_measures_values'])

  arr_general_measures_names = []
  arr_general_measures_values = []
  arr_additional_measures_names = []
  arr_additional_measures_values = []
  arr_additional_tw_measures_names = []
  arr_additional_tw_measures_values = []

  first_comma_pos = 0
  last_comma_pos = 0

  first_comma_found = False
  last_comma_found = False

  len_gm = len(general_measures_names)
  len_adm = len(additional_measures_names)
  len_admtw = len(additional_tw_measures_names)

  count_gm = 0
  count_adm = 0
  count_admtw = 0


  # split general_measures_names
  while count_gm < len_gm:
    i = count_gm

    if general_measures_names.find(',', pos=i, count=1) >= i:
      if not first_comma_found:
        first_comma_pos = i
        first_comma_found = True
      else:
        last_comma_pos = i
        last_comma_found =  True

    if first_comma_found and last_comma_found:
      piece = '@'
      start = first_comma_pos + 1
      end = last_comma_pos + 1
      r = start

      while r <= end:
        piece += general_measures_names[r]
        r += 1

      arr_general_measures_names.append(piece)
      first_comma_found = False
      last_comma_found = False

    count_gm += 1


  # split additional_measures_names
  while count_adm < len_adm:
    i = count_adm

    if additional_measures_names.find(',', pos=i, count=1) >= i:
      if not first_comma_found:
        first_comma_pos = i
        first_comma_found = True
      else:
        last_comma_pos = i
        last_comma_found =  True

    if first_comma_found and last_comma_found:
      piece = '@'
      start = first_comma_pos + 1
      end = last_comma_pos + 1
      r = start

      while r <= end:
        piece += additional_measures_names[r]
        r += 1

      arr_additional_measures_names.append(piece)
      first_comma_found = False
      last_comma_found = False

    count_adm += 1


  # split additional_tw_measures_names
  while count_admtw < len_admtw:
    i = count_admtw

    if additional_tw_measures_names.find(',', pos=i, count=1) >= i:
      if not first_comma_found:
        first_comma_pos = i
        first_comma_found = True
      else:
        last_comma_pos = i
        last_comma_found =  True

    if first_comma_found and last_comma_found:
      piece = '@'
      start = first_comma_pos + 1
      end = last_comma_pos + 1
      r = start

      while r <= end:
        piece += additional_tw_measures_names[r]
        r += 1

      arr_additional_tw_measures_names.append(piece)
      first_comma_found = False
      last_comma_found = False

    count_admtw += 1


  count_gm = 0
  count_adm = 0
  count_admtw = 0

  #replace general_measures
  while count_gm < len_gm:
    i = count_gm

    if replaced_formula.contains(arr_general_measures_names[i]):
      replaced_formula = replaced_formula.replace(arr_additional_measures_names[i], arr_additional_measures_values[i])

    count_gm += 1


  #replace additional_measures_names
  while count_adm < len_adm:
    i = count_adm

    if replaced_formula.contains(arr_additional_measures_names[i]):
      replaced_formula = replaced_formula.replace(arr_additional_measures_names[i], arr_additional_measures_values[i])

    count_adm += 1

  #replace additional_tw_measures_names
  while count_admtw < len_admtw:
    i = count_admtw

    if replaced_formula.contains(arr_additional_tw_measures_names[i]):
      replaced_formula = replaced_formula.replace(arr_additional_tw_measures_names[i], arr_additional_tw_measures_values[i])

    count_admtw += 1


  return replaced_formula


#df['Formula_USD_replaced'] = df.apply(replace_formula_usd_kernel, axis=1)

In [48]:
df_additional_measures = pd.read_csv('/content/gpu_data/ADDITIONAL_MEASURES.csv', ';')
df_additional_tw_measures = pd.read_csv('/content/gpu_data/ADDITIONAL_TOWERCO_MEASURES.csv', ';')
df_exchange_rates = pd.read_csv('/content/gpu_data/EXCHANGE_RATE.csv', ';')
df_general_measures = pd.read_csv('/content/gpu_data/GENERAL_MEASURES.csv', ';')
df_leg_for_lookup_codes = pd.read_csv('/content/gpu_data/LEGEND_FOR_LOOKUP_CODES.csv', ';')

joint_gm_names = ','.join(list(df_general_measures['MEASURE_TYPE'].astype(str).values_host))
joint_gm_values = ','.join(list(df_general_measures['MEASURE_VALUE'].astype(str).values_host))
joint_am_names = ','.join(list(df_additional_measures['MEASURE_TYPE'].astype(str).values_host))
joint_am_values = ','.join(list(df_additional_measures['MEASURE_VALUE'].astype(str).values_host))
joint_tw_names = ','.join(list(df_additional_tw_measures['MEASURE_TYPE'].astype(str).values_host))
joint_tw_values = ','.join(list(df_additional_tw_measures['MEASURE_VALUE'].astype(str).values_host))

df['general_measures_names'] = joint_gm_names
df['general_measures_values'] = joint_gm_values
df['additional_measures_names'] = joint_am_names
df['additional_measures_values'] = joint_am_values
df['additional_tw_measures_names'] = joint_tw_names
df['additional_tw_measures_values'] = joint_tw_values



def replace_formula_usd_kernel(
    Formula_USD,
    general_measures_names_array,
    general_measures_values_array,
    additional_measures_names_array,
    additional_measures_values_array,
    additional_tw_measures_names_array,
    additional_tw_measures_values_array
):
  replaced_formulas = []

  for idx, formula in enumerate(Formula_USD):
    replaced_formula = formula.as_py()

    arr_general_measures_names = []
    arr_general_measures_values = []
    arr_additional_measures_names = []
    arr_additional_measures_values = []
    arr_additional_tw_measures_names = []
    arr_additional_tw_measures_values = []

    first_comma_pos = 0
    last_comma_pos = 0

    first_comma_found = False
    last_comma_found = False

    general_measures_names = general_measures_names_array[idx].as_py()
    additional_measures_names = additional_measures_names_array[idx].as_py()
    additional_tw_measures_names = additional_tw_measures_names_array[idx].as_py()

    len_gm = len(general_measures_names)
    len_adm = len(additional_measures_names)
    len_admtw = len(additional_tw_measures_names)

    count_gm = 0
    count_adm = 0
    count_admtw = 0


    # split general_measures_names
    while count_gm < len_gm:
      i = count_gm

      if general_measures_names[i] == ',':
        if not first_comma_found:
          first_comma_pos = i
          first_comma_found = True
        else:
          last_comma_pos = i
          last_comma_found =  True

      if first_comma_found and last_comma_found:
        start = first_comma_pos + 1
        end = last_comma_pos + 1
        piece = '@' + general_measures_names[start:end]

        arr_general_measures_names.append(piece)
        first_comma_found = False
        last_comma_found = False

      count_gm += 1


    # split additional_measures_names
    while count_adm < len_adm:
      i = count_adm

      if additional_measures_names[i] == ',':
        if not first_comma_found:
          first_comma_pos = i
          first_comma_found = True
        else:
          last_comma_pos = i
          last_comma_found =  True

      if first_comma_found and last_comma_found:
        start = first_comma_pos + 1
        end = last_comma_pos + 1
        piece = '@' + additional_measures_names[start:end]

        arr_additional_measures_names.append(piece)
        first_comma_found = False
        last_comma_found = False

      count_adm += 1


    # split additional_tw_measures_names
    while count_admtw < len_admtw:
      i = count_admtw

      if additional_tw_measures_names[i] == ',':
        if not first_comma_found:
          first_comma_pos = i
          first_comma_found = True
        else:
          last_comma_pos = i
          last_comma_found =  True

      if first_comma_found and last_comma_found:
        start = first_comma_pos + 1
        end = last_comma_pos + 1
        piece = '@' + additional_tw_measures_names[start:end]

        arr_additional_tw_measures_names.append(piece)
        first_comma_found = False
        last_comma_found = False

      count_admtw += 1


    #replace general_measures
    for i in range(len(arr_general_measures_names)):
      if arr_general_measures_names[i] in replaced_formula:
        replaced_formula = replaced_formula.replace(
            arr_general_measures_names[i],
            arr_general_measures_values[i]
        )


    #replace additional_measures_names
    for i in range(len(arr_additional_measures_names)):
      if arr_additional_measures_names[i] in replaced_formula:
        replaced_formula = replaced_formula.replace(
            arr_additional_measures_names[i],
            arr_additional_measures_values[i]
        )

    #replace additional_tw_measures_names
    for i in range(len(arr_additional_tw_measures_names)):
      if arr_additional_tw_measures_names[i] in replaced_formula:
        replaced_formula = replaced_formula.replace(
            arr_additional_tw_measures_names[i],
            arr_additional_tw_measures_values[i]
        )

    replaced_formulas[idx] = replaced_formula

  return replaced_formulas


df['Formula_USD_replaced'] = replace_formula_usd_kernel(
    df['Formula_USD'].to_arrow(),
    df['general_measures_names'].to_arrow(),
    df['general_measures_values'].to_arrow(),
    df['additional_measures_names'].to_arrow(),
    df['additional_measures_values'].to_arrow(),
    df['additional_tw_measures_names'].to_arrow(),
    df['additional_tw_measures_values'].to_arrow()
)

IndexError: list index out of range